In [ ]:
# Bibliotecas necessárias
import bibtexparser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
import pandas as pd
import numpy as np
import os
import requests
doi_headers = {'Accept': 'text/bibliography; style=bibtex'}
doi = '10.23919/CYCON.2019.8756895'

In [ ]:
# Funções
def parser_bibtex(file_bib):
    with open(file_bib,encoding='utf-8') as bibtex_file:
        try:
            bib_database = bibtexparser.load(bibtex_file)
            return pd.DataFrame(bib_database.entries)
        except:
            print('Falha no arquivo', file_bib)
            return pd.DataFrame(None)

def info_doi(doi):
    try:
        response = requests.get('http://dx.doi.org/'+doi, headers=doi_headers)
        bib_database = bibtexparser.loads(response.text)
        return pd.DataFrame.from_dict(bib_database.entries)
    except:
        print(doi,'\tOcorreu um erro ao pesquisar!')

def journal_to_inproceedings(journal):
    x = []
    x.extend([journal.split(' ')[0],journal.split(' ')[1]])
    x.append(journal.split(x[0]+' '+x[1]+' ')[1])
    # 'conference-year', 'conference-number', 'conference-location'

# Depois de geradas as referências automaticamente ou mesmo adicionadas manualmente, como consultas em sites e outros trabalhos de Dissertação ou Tese, será necessária a correção destas referências para os campos adequados para cada tipo de entrada, conforme Norma ABNT.

# Gerar Referências padrão ITA para implementação no LATEX

In [ ]:
# Campos Obrigatórios
dados = {
    'article': ['ENTRYTYPE','ID','year', 'pages', 'volume', 'journal', 'title','author'],
    'book': ['ENTRYTYPE','ID','year','pages', 'volume', 'title',
       'author', 'ENTRYTYPE', 'ID', 'url', 'publisher', 'issn',
       'isbn', 'address', 'booktitle'],
    'inproceedings': ['ENTRYTYPE','ID','address','author','conference-location','conference-number','conference-year',
        'organization','pages','publisher','subtitle','title','url','year'],
    'lei': ['ENTRYTYPE','ID','address','journal','month','organization','volume','title','number','pages','url','urlaccessdate','year'],
    'mastersthesis': ['ENTRYTYPE','ID','author', 'pages', 'title', 'year', 'year-presented', 'type', 'publisher', 'school', 'address'],
    'misc': ['ENTRYTYPE','ID','address','author','edition','note','publisher','subtitle','title','url','urlaccessdate','volume','year'],
    'techreport': ['ENTRYTYPE','ID','address','author','booktitle','edition','isbn','pages','publisher','subtitle','title','url','volume','year']
}

In [ ]:
ref = 'referencias_mapeamento.bib'
df = parser_bibtex(ref)

In [ ]:
for entry in (dados.keys()):
    globals()['df_' + entry] = df.loc[(entry == df['ENTRYTYPE'])].drop(columns=['abstract','keywords'],axis=1)
    globals()['df_' + entry].reset_index(inplace=True,drop=True)
    globals()['df_' + entry] = globals()['df_' + entry].replace(np.nan, '', regex=True)
    globals()[entry] = globals()['df_' + entry].to_dict(orient="records")
    globals()['db_' + entry] = BibDatabase()
    globals()['db_' + entry].entries = globals()[entry]

    if(entry == 'inproceedings'):
        df['conference-year', 'conference-number', 'conference-location'] = journal_to_inproceedings(df['journal'])
        #resolver conferência
        #remover journal
    with open('referencias.bib', 'a',encoding='utf-8') as bibfile:
        bibfile.write(writer.write(globals()['db_' + entry]))
    
#    del globals()['df_' + entry], globals()['db_' + entry],globals()[entry]
#del writer, bibfile